# Pendahuluan

Analisis ini bertujuan untuk mengklasifikasikan diagnosis kanker payudara (Malignant atau Benign)
berdasarkan dataset *Breast Cancer Wisconsin Diagnostic* menggunakan algoritma *K-Nearest Neighbors (K-NN)*.
Selain itu, dilakukan *Feature Selection* dengan Principal Component Analysis (PCA) untuk mengurangi dimensi fitur.

# 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

#  2. Import Dataset

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving wdbc_data.csv to wdbc_data.csv


In [ ]:
file_path = 'wdbc_data.csv'
headers = [
    "ID", "Diagnosis", "radius1", "texture1", "perimeter1", "area1", "smoothness1", "compactness1", "concavity1", "concave_points1", "symmetry1", "fractal_dimension1",
    "radius2", "texture2", "perimeter2", "area2", "smoothness2", "compactness2", "concavity2", "concave_points2", "symmetry2", "fractal_dimension2",
    "radius3", "texture3", "perimeter3", "area3", "smoothness3", "compactness3", "concavity3", "concave_points3", "symmetry3", "fractal_dimension3"
]

df = pd.read_csv(file_path, header=None, names=headers)

In [ ]:
df.head()

,ID,Diagnosis,radius1,texture1,perimeter1,area1,smoothness1,compactness1,concavity1,concave_points1,...,radius3,texture3,perimeter3,area3,smoothness3,compactness3,concavity3,concave_points3,symmetry3,fractal_dimension3
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


* Kolom pertama (ID) tidak diperlukan untuk analisis dan akan dihapus.
* Kolom kedua (Diagnosis) adalah target label (M = Malignant, B = Benign), akan dikonversi ke numerik.
* Semua fitur lainnya akan dinormalisasi agar memiliki skala yang sama.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  569 non-null    int64  
 1   Diagnosis           569 non-null    object 
 2   radius1             569 non-null    float64
 3   texture1            569 non-null    float64
 4   perimeter1          569 non-null    float64
 5   area1               569 non-null    float64
 6   smoothness1         569 non-null    float64
 7   compactness1        569 non-null    float64
 8   concavity1          569 non-null    float64
 9   concave_points1     569 non-null    float64
 10  symmetry1           569 non-null    float64
 11  fractal_dimension1  569 non-null    float64
 12  radius2             569 non-null    float64
 13  texture2            569 non-null    float64
 14  perimeter2          569 non-null    float64
 15  area2               569 non-null    float64
 16  smoothne

Bisa diliat jika tidak ada missing value pada semua kolom dimana bertotalkan 569 row / data

In [ ]:
df.describe()

,ID,radius1,texture1,perimeter1,area1,smoothness1,compactness1,concavity1,concave_points1,symmetry1,...,radius3,texture3,perimeter3,area3,smoothness3,compactness3,concavity3,concave_points3,symmetry3,fractal_dimension3
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


# 3. Preprocessing

## Hapus Kolom yang tidak terpakai

In [ ]:
df.drop(columns=["ID"], inplace=True)

## Mengubah label menjadi numerik

In [ ]:
label_encoder = LabelEncoder()
df["Diagnosis"] = label_encoder.fit_transform(df["Diagnosis"])

Melakukan pengubahan menjadi numerik terhadap kolom diagnosis (M = Malignant, B = Benign) dimana akan mengubah nilai
* M menjadi 1
* B menjadi 0


# Persiapan untuk feature Selection

In [ ]:
X = df.iloc[:, 1:]
y = df["Diagnosis"].astype(int)

Dari sini terdapat pemisahan dimana
* Y hanya berisi target label yaitu kolom Diagnosis
* X mengambil semua kolom kecuali Diagnosis

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Semua kolom yang ada di X melakukan normalisasi supaya pembagian angka tersebut merata dengan tujuan meningkatkan efisiensi, mengurangi redundansi data, dan meningkatkan akurasi data.

# 4. Feature Selection dengan PCA

* PCA digunakan untuk mereduksi dimensi data tanpa kehilangan informasi penting.
* Digunakan 10 komponen utama berdasarkan eksperimen.

In [ ]:
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X_scaled)

Kemudian kita akan melihat proporsi varians yang dijelaskan oleh setiap komponen
 dalam X

In [ ]:
print("Explained Variance Ratio:", pca.explained_variance_ratio_)

Explained Variance Ratio: [0.44272026 0.18971182 0.09393163 0.06602135 0.05495768 0.04024522
 0.02250734 0.01588724 0.01389649 0.01168978]


# 5. Model Klasifikasi dengan K-NN dan Cross-Validation

* Model K-NN digunakan untuk klasifikasi dengan k=5.
* Cross-validation 5-fold dilakukan untuk evaluasi stabilitas model.

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
cv_scores = cross_val_score(knn, X_pca, y, cv=5)

In [ ]:
print(f"Cross-Validation Accuracy: {np.mean(cv_scores) * 100:.2f}% ± {np.std(cv_scores) * 100:.2f}%")

Cross-Validation Accuracy: 96.13% ± 1.32%


Dengan melakukan Cross Validation menggunakan model K-Nearest Neighbors didapatkan hasil akurasi sekitar antara 94,81% - 97,45% dengan pusatnya berada di 96.13%

# 6. Evaluasi Model dengan Data Test

* Data dibagi menjadi training 80% dan testing 20%.
* Model K-NN dilatih dengan data training dan diuji dengan data test.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

In [ ]:
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [ ]:
y_pred = knn.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Set Accuracy: {accuracy * 100:.2f}%')
print("Classification Report:")
print(classification_report(y_test, y_pred))

Analisis Hasil:
1. Model K-NN mendapatkan akurasi sebesar 96.13% pada *Cross-Validation*.
2. Dengan data uji, akurasi model adalah 95.61%.
3. PCA membantu mengurangi dimensi data, tetapi tetap mempertahankan informasi yang cukup untuk klasifikasi.
4. Model K-NN bekerja cukup baik dalam membedakan antara kanker *Malignant* dan *Benign*.